In [19]:
import os
import time
from dateutil.parser import parse as dateutil_parse
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler as skStandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.cross_validation import train_test_split,cross_val_score

PATH = 'season_1/'
CLEAN_PATH = PATH+'clean/'

from mylib import myStandardScaler,process_order,process_traffic,process_weather,get_order_group,get_traffic_group,XY_order_traffic, prediction2submit, metrics, DISTRICTS

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
cluster_map = pd.read_csv(CLEAN_PATH+'cluster_map.csv',index_col=0)
poi = pd.read_csv(CLEAN_PATH+'poi.csv',index_col=0)
train_order = pd.read_pickle(CLEAN_PATH+'train_order.pickle')
test_order = pd.read_pickle(CLEAN_PATH+'test_order.pickle')
train_traffic = pd.read_pickle(CLEAN_PATH+'train_traffic.pickle')
test_traffic = pd.read_pickle(CLEAN_PATH+'test_traffic.pickle')
train_weather = pd.read_pickle(CLEAN_PATH+'train_weather.pickle')
test_weather = pd.read_pickle(CLEAN_PATH+'test_weather.pickle')
test_target = pd.read_csv(CLEAN_PATH+'test_target.csv',index_col=0,parse_dates=True)

In [12]:
now = time.time()

In [13]:
train_order = process_order(train_order)
test_order = process_order(test_order)
train_traffic = process_traffic(train_traffic)
test_traffic = process_traffic(test_traffic)

In [14]:
train_order_group = get_order_group(train_order)
test_order_group = get_order_group(test_order)
train_traffic_group = get_traffic_group(train_traffic)
test_traffic_group = get_traffic_group(test_traffic)

In [15]:
train_slot = pd.Index(sorted(train_order['datetimeslot'].unique()))
train_slot = pd.Index(filter(lambda x: x%1000 >4,train_slot))
test_slot = test_target['datetimeslot']

In [16]:
print 'Take %.2fs' % (time.time()-now)

Take 1064.16s


In [45]:
train_order_group.values()[0].ix[filter(lambda x: x%1000==100,train_order_group.values()[0].index)].shape # too litte for 1 hour

(21, 20)

In [117]:
tmp = process_weather(test_weather).groupby('datetimeslot').mean()
tmp_ = (tmp.ix[test_slot-3].rename(index=lambda x: x + 3).notnull().applymap(lambda x: int(x)) + \
tmp.ix[test_slot-2].rename(index=lambda x: x + 2).notnull().applymap(lambda x: int(x)) + \
tmp.ix[test_slot-1].rename(index=lambda x: x + 1).notnull().applymap(lambda x: int(x)))
((tmp.ix[test_slot-3].rename(index=lambda x: x + 3).fillna(0) + \
tmp.ix[test_slot-2].rename(index=lambda x: x + 2).fillna(0) + \
tmp.ix[test_slot-1].rename(index=lambda x: x + 1).fillna(0))/tmp_.replace(0,np.nan))
tmp.ix[test_slot]

,weather_1,weather_2,weather_3,weather_4,weather_6,weather_8,weather_9,temperature,pm25,timeslot
datetimeslot,,,,,,,,,,
20160122046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20160122058,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20160122070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20160122082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20160122094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20160122106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20160122118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20160122130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20160122142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
tmp = tmp.groupby('datetimeslot').mean().fillna(method='ffill')
(tmp.ix[test_slot-3].rename(index=lambda x: x + 3) + tmp.ix[test_slot-2].rename(index=lambda x: x + 2) + tmp.ix[test_slot-1].rename(index=lambda x: x + 1))/3.

KeyError: 'datetimeslot'

In [98]:
test_weather.ix['20160124']

,weather,temperature,pm25,timeslot,datetimeslot
datatime,,,,,
2016-01-24 11:12:11,1,-6.0,63,68,20160124068
2016-01-24 13:16:58,1,-4.0,61,80,20160124080
2016-01-24 15:06:53,1,-4.0,54,91,20160124091
2016-01-24 15:21:52,1,-4.0,54,93,20160124093
2016-01-24 17:01:37,1,-4.0,50,103,20160124103
2016-01-24 17:22:19,1,-5.0,50,105,20160124105
2016-01-24 19:07:06,2,-6.0,51,115,20160124115
2016-01-24 21:13:01,1,-6.0,58,128,20160124128
2016-01-24 23:02:15,1,-7.0,59,139,20160124139


In [118]:
tmp = process_weather(train_weather).groupby('datetimeslot').mean()
tmp_ = (tmp.ix[train_slot-3].rename(index=lambda x: x + 3).notnull().applymap(lambda x: int(x)) + \
tmp.ix[train_slot-2].rename(index=lambda x: x + 2).notnull().applymap(lambda x: int(x)) + \
tmp.ix[train_slot-1].rename(index=lambda x: x + 1).notnull().applymap(lambda x: int(x)))
res = ((tmp.ix[train_slot-3].rename(index=lambda x: x + 3).fillna(0) + \
tmp.ix[train_slot-2].rename(index=lambda x: x + 2).fillna(0) + \
tmp.ix[train_slot-1].rename(index=lambda x: x + 1).fillna(0))/tmp_.replace(0,np.nan))
res.to_csv('train_weather.csv')
# len(set(res.notnull()[res.notnull()].dropna().index.map(lambda x: x%1000)))

In [100]:
process_weather(len)

,weather_1,weather_2,weather_3,weather_4,weather_6,weather_8,weather_9,temperature,pm25,timeslot,datetimeslot
datatime,,,,,,,,,,,
2016-01-01 00:00:28,True,False,False,False,False,False,False,4.0,177,1,20160101001
2016-01-01 00:05:24,True,False,False,False,False,False,False,3.0,177,1,20160101001
2016-01-01 00:10:08,True,False,False,False,False,False,False,3.0,177,2,20160101002
2016-01-01 00:15:27,True,False,False,False,False,False,False,3.0,177,2,20160101002
2016-01-01 00:20:06,True,False,False,False,False,False,False,3.0,177,3,20160101003
2016-01-01 00:25:25,True,False,False,False,False,False,False,3.0,177,3,20160101003
2016-01-01 00:30:08,True,False,False,False,False,False,False,3.0,177,4,20160101004
2016-01-01 00:35:21,True,False,False,False,False,False,False,3.0,177,4,20160101004
2016-01-01 00:40:04,True,False,False,False,False,False,False,3.0,177,5,20160101005


In [40]:
for weather in train_weather.weather.unique():
    print weather, (train_weather.weather==weather).mean()*100

1 3.57514030347
8 6.71378091873
2 45.7285387653
9 0.436499688215
3 16.3583454583
4 27.1876948659


In [41]:
for weather in test_weather.weather.unique():
    print weather, (test_weather.weather==weather).mean()*100

2 19.2307692308
4 32.0512820513
6 3.84615384615
1 29.4871794872
9 2.5641025641
3 12.8205128205
